In [4]:
import os
import open3d as o3d
import shutil
import trimesh
import numpy as np
from utils import pc2mesh, pc2mesh_v2, compute_pc_mesh_dim_ratio, process_mesh, natural_keys

In [3]:
pc_source_folder = "assets/ground_truth_5000_correct_unit"
mesh_save_folder = "assets/ground_truth_5000_correct_unit_mesh"
mesh_source_folder = "assets/mesh_correct_pose"
os.makedirs(mesh_save_folder, exist_ok=True)

obj_basenames = os.listdir(pc_source_folder)
for obj_basename in obj_basenames:
    obj_name = obj_basename.split(".")[0]
    pc_path = os.path.join(pc_source_folder, obj_basename)
    mesh_file_path = os.path.join(mesh_source_folder, f"{obj_name}.ply")
    if not os.path.exists(mesh_file_path):
        continue
    print(compute_pc_mesh_dim_ratio(pc_path, mesh_file_path))

FileNotFoundError: [Errno 2] No such file or directory: 'assets/ground_truth_5000_correct_unit'

In [5]:
# Scale mesh to unit size and save
mesh_source_folder = "assets/mesh_correct_pose"
unit_mesh_save_folder = "assets/mesh_correct_pose_unit"
os.makedirs(unit_mesh_save_folder, exist_ok=True)
obj_meshes = sorted(os.listdir(mesh_source_folder), key=natural_keys)

for obj_mesh in obj_meshes:
    print(f"Processing {obj_mesh}")
    obj_mesh_path = os.path.join(mesh_source_folder, obj_mesh)
    obj_mesh_save_path = os.path.join(unit_mesh_save_folder, obj_mesh)
    try:
        process_mesh(obj_mesh_path, obj_mesh_save_path, scale_factor=[1e-3, 1e-3, 1e-3], rotate_angle=[0., 0., 0])
    except Exception as e:
        print(f"Error: {e}")
        continue

Processing 100_wood_cooking_spoon.ply
Processing 101_wood_instrument.ply
Processing 102_plastic_fruit_pear.ply
Processing 103_plastic_fruit_apple.ply
Processing 104_plastic_fruit_mango.ply
Processing 105_plastic_fruit_banana.ply
Processing 106_tissue_box.ply
Processing 107_cuboid_rubber.ply
Processing 108_triangular_pyramid_rubber.ply
Processing 109_cone_rubber.ply
Processing 10_coca_colar.ply
Processing 110_quadrangular_pyramid_rubber.ply
Processing 111_quadrangular_pyramid_foam.ply
Processing 112_cylinder_foam.ply
Processing 113_wood_jar_1.ply
Processing 114_wood_jar_2.ply
Processing 115_wood_jar_3.ply
Processing 116_hard_paper_box.ply
Processing 117_wood_jar_without_lid_1.ply
Processing 118_wood_jar_without_lid_2.ply
Processing 119_wood_scoop.ply
Processing 11_coffee_container_paper.ply
Processing 120_cooking_pot.ply
Processing 121_triangular_pyramid_fabric.ply
Processing 122_cone_fabric.ply
Processing 123_cuboid_fabric.ply
Processing 124_triangular_prism_fabric.ply
Processing 125_w

In [1]:
import open3d as o3d
import os

mesh_folder = "assets/group_objects/group4_real_objects_mesh"
save_as_obj = True
downsampled_mesh_folder = "assets/group_objects/group4_real_objects_mesh_downsampled_obj" if save_as_obj else "assets/group_objects/group4_real_objects_mesh_downsampled"
os.makedirs(downsampled_mesh_folder, exist_ok=True)
mesh_files = os.listdir(mesh_folder)
for mesh_name in mesh_files:
    mesh_path = os.path.join(mesh_folder, mesh_name)
    mesh_in = o3d.io.read_triangle_mesh(mesh_path, True)
    mesh_in.compute_vertex_normals()
    print(
        f'Input mesh {mesh_name} has {len(mesh_in.vertices)} vertices and {len(mesh_in.triangles)} triangles'
    )
    print(f"{mesh_in.textures}")
    if len(mesh_in.triangles) > 10000:
        mesh_out = mesh_in.simplify_quadric_decimation(target_number_of_triangles=10000)
    else:
        mesh_out = mesh_in
    # save downsampled mesh to .obj file
    if save_as_obj:
        mesh_out_path = os.path.join(downsampled_mesh_folder, mesh_name.replace(".ply", ".obj"))
    else:
        mesh_out_path = os.path.join(downsampled_mesh_folder, mesh_name)
    o3d.io.write_triangle_mesh(mesh_out_path, mesh_out)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Input mesh 133_domino_suger.ply has 117558 vertices and 235112 triangles
[]
[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
Input mesh 34_meyerscleanday.ply has 74812 vertices and 149616 triangles
[]
[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
Input mesh 5_chinese_ceramic_bowl_large.ply has 237832 vertices and 475660 triangles
[]
[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
Input mesh 36_mustard.ply has 124828 vertices and 249652 triangles
[]
[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
Input mesh 44_paper_tape.ply has 232258 vertices and 464516 triangles
[]
[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
Input mesh 45_pepper_powder.ply has 120940 vertices and 24

In [20]:
# visualize
downsampled_mesh_files = os.listdir(downsampled_mesh_folder)
for mesh_name in downsampled_mesh_files:
    mesh_path = os.path.join(downsampled_mesh_folder, mesh_name)
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    o3d.visualization.draw_geometries([mesh])

In [23]:
# # Create object folder and urdf for each mesh file

# YCB dataset requires to set visual to be white. The default is black which will block the texture.
def create_urdf(robot_name, visual_mesh_filename, collision_mesh_filename, 
                mass=0.1, scale=[1, 1, 1.], origin_xyz=[0., 0., 0.], 
                inertia_ixx="0.0001", inertia_ixy="0.0", inertia_ixz="0.0", 
                inertia_iyy="0.0001", inertia_iyz="0.0", inertia_izz="0.0001", save_path=None):
    """
    Creates a URDF file as a string based on the given parameters.

    :param robot_name: Name of the robot.
    :param visual_mesh_filename: Filename of the visual mesh.
    :param collision_mesh_filename: Filename of the collision mesh.
    :param mass: Mass of the link.
    :param origin_xyz: Origin offset, default "0.0 0.0 0.0".
    :param inertia_ixx, inertia_ixy, inertia_ixz, inertia_iyy, inertia_iyz, inertia_izz: Inertia parameters.
    :return: A string representing the URDF file.
    """
    
    urdf_template = \
f"""<?xml version='1.0' encoding='utf-8'?>
    <robot name="{robot_name}">
        <link name="link_0">
            <visual>
                <origin xyz="{' '.join(map(str, origin_xyz))}" />
                <geometry>
                    <mesh filename="{visual_mesh_filename}" scale="{' '.join(map(str, scale))}" />
                </geometry>
                <material name="white">
                    <color rgba="1 1 1 1"/>
                </material>
            </visual>
            <collision>
                <origin xyz="{' '.join(map(str, origin_xyz))}" />
                <geometry>
                    <mesh filename="{collision_mesh_filename}" scale="{' '.join(map(str, scale))}" />
                </geometry>
            </collision>
            <inertial>
                <mass value="{mass}" />
                <inertia ixx="{inertia_ixx}" ixy="{inertia_ixy}" ixz="{inertia_ixz}" 
                            iyy="{inertia_iyy}" iyz="{inertia_iyz}" izz="{inertia_izz}" />
            </inertial>
        </link>
    </robot>"""
    if save_path:
        with open(save_path, 'w') as file:
            file.write(urdf_template)
        print(f"URDF file saved to {save_path}")

    return urdf_template

mesh_source_folder = "assets/group_objects/group4_real_objects_mesh_downsampled_obj"
obj_categories = sorted(os.listdir(mesh_source_folder))
dataset_target_folder = "assets/group_objects/group4_real_objects"
os.makedirs(dataset_target_folder, exist_ok=True)

failed_objs = []
for obj_mesh in obj_categories:
  cate = obj_mesh.split(".")[0]
  cate_target_folder = os.path.join(dataset_target_folder, cate)
  obj_target_folder = os.path.join(cate_target_folder, "0") # We now only have one object per category for YCB
  textured_objs_folder = os.path.join(obj_target_folder, "textured_objs")
  os.makedirs(textured_objs_folder, exist_ok=True)
  # Convert ply to obj
  obj_mesh_path = os.path.join(mesh_source_folder, obj_mesh)
  obj_target_mesh_path = os.path.join(textured_objs_folder, "textured.obj")
  mesh = trimesh.load(obj_mesh_path)
  mesh.export(obj_target_mesh_path)

  # Generate urdf
  urdf_path = os.path.join(obj_target_folder, "mobility.urdf")
  # Mesh file path are all relative to the urdf file!!!
  create_urdf(robot_name=f"{cate}_0", 
              visual_mesh_filename="textured_objs/textured.obj", 
              collision_mesh_filename="textured_objs/textured.obj",
              save_path=urdf_path)

URDF file saved to assets/group_objects/group4_real_objects/125_wine_glass/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/131_large_glass_cup/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/133_domino_suger/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/23_dawn/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/34_meyerscleanday/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/36_mustard/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/37_M_mug/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/44_paper_tape/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/45_pepper_powder/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/46_pepsi/0/mobility.urdf
URDF file saved to assets/group_objects/group4_real_objects/5_chinese_ceramic_bowl_large/0/mobility.ur

# Flat the bottom of mesh to be a flatten surface

In [24]:
import trimesh
import os

group_folder = "assets/group_objects/group4_real_objects"
obj_categories = sorted(os.listdir(group_folder))
for obj_cate in obj_categories:
    cate_folder = os.path.join(group_folder, obj_cate)
    obj_folders = sorted(os.listdir(cate_folder))
    for obj_folder in obj_folders:
        textured_obj_folder = os.path.join(cate_folder, obj_folder, "textured_objs")
        obj_path = os.path.join(textured_obj_folder, "textured.obj")
        mesh = trimesh.load(obj_path)
        min_z = mesh.vertices[:, 2].min()
        # Define a tolerance level for the bottom area (e.g., vertices that are within 1mm of the min_z)
        tolerance = 0.005

        # Create a mask for all vertices that are within the tolerance range of the minimum Z
        bottom_vertices_mask = mesh.vertices[:, 2] <= (min_z + tolerance)

        # Set the Z-coordinate of these vertices to min_z
        mesh.vertices[bottom_vertices_mask, 2] = min_z
        mesh.export(obj_path)
        mesh.show()


# Generate Primitive Mesh

In [ ]:
import trimesh
import os

